### Simpler Model: 1D Convolutional Neural Network (CNN)

To complement the baseline STFT + Conv2D + RNN model, we implemented a lightweight 1D CNN that operates directly on raw ECG signals. This model is more straightforward, easier to train, and more stable in early training phases.

#### Architecture Overview
The model consists of a stack of three 1D convolutional blocks followed by a global average pooling and a fully connected classifier:
- Conv1d → BatchNorm1d → ReLU → MaxPool1d
- Conv1d → BatchNorm1d → ReLU → MaxPool1d
- Conv1d → BatchNorm1d → ReLU → AdaptiveAvgPool1d
- Linear → output logits for 4 classes

Each raw ECG signal is zero-padded to the length of the longest signal in the batch to ensure compatibility with batch processing.

#### Why 1D CNN?
1D CNNs are well-suited for time series tasks like ECG classification due to their ability to extract local patterns (e.g., QRS complexes) efficiently across the signal. Unlike the baseline, this model avoids frequency-domain transformation and recurrent layers, resulting in:
- Faster training
- Lower complexity
- Competitive performance, especially with imbalanced data

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
from src.ecg_dataset import ECGDataset, prep_batch
from src.cnn1d import CNN1DModel
from src.train_utils import train_one_epoch, evaluate
from src.train import train_model
from src.parser import read_zip_binary

# load data (same as before)
train_idx = np.load("../data/train_idx.npy")
val_idx = np.load("../data/val_idx.npy")

X_train = read_zip_binary("../data/X_train.zip")
y_train = pd.read_csv("../data/y_train.csv", header=None)
y_train.columns = ["y"]

train_dataset = ECGDataset(X_train, y_train, indices=train_idx)
val_dataset = ECGDataset(X_train, y_train, indices=val_idx)

# prepare data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=prep_batch)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=prep_batch)

# device is cpu by default but added this line (instead of hardcoding cpu string) since you may be using cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# weights are added to counter class imbalance
# idea is basically penalizing the rare classes more in order to classify them correctly
weights = compute_class_weight(class_weight="balanced", classes=[0, 1, 2, 3], y=y_train["y"])
weights = torch.tensor(weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights)

# load the 1d cnn model into device -- similar practices as always
model = CNN1DModel(n_classes=4).to(device)
# TODO : maybe we can work more on learning rate and find a good balance between lr and # of epochs
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-3)

# enter the training loop -- train_model returns the best model in the end
model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    device=device,
    num_epochs=20
)

/Users/cuzdu/Desktop/early-alzhemir-detection/env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01 | Time: 208.3s
  Train Loss: 1.3108 | Acc: 0.3251 | F1: 0.2754
  Val   Loss: 1.2417 | Acc: 0.3244 | F1: 0.2342
Epoch 02 | Time: 256.7s
  Train Loss: 1.2176 | Acc: 0.3585 | F1: 0.2943
  Val   Loss: 1.1852 | Acc: 0.3083 | F1: 0.2619
Epoch 03 | Time: 237.1s
  Train Loss: 1.1396 | Acc: 0.3779 | F1: 0.3196
  Val   Loss: 1.1582 | Acc: 0.3252 | F1: 0.3055
Epoch 04 | Time: 220.7s
  Train Loss: 1.1171 | Acc: 0.3759 | F1: 0.3291
  Val   Loss: 1.1204 | Acc: 0.5526 | F1: 0.3832
Epoch 05 | Time: 243.2s
  Train Loss: 1.1039 | Acc: 0.3868 | F1: 0.3317
  Val   Loss: 1.5044 | Acc: 0.2961 | F1: 0.2528
Epoch 06 | Time: 250.8s
  Train Loss: 1.0864 | Acc: 0.4111 | F1: 0.3500
  Val   Loss: 1.1038 | Acc: 0.2888 | F1: 0.2973
Epoch 07 | Time: 221.2s
  Train Loss: 1.0716 | Acc: 0.4099 | F1: 0.3523
  Val   Loss: 1.0887 | Acc: 0.4474 | F1: 0.3605
Epoch 08 | Time: 229.8s
  Train Loss: 1.0386 | Acc: 0.4439 | F1: 0.3693
  Val   Loss: 1.0747 | Acc: 0.3932 | F1: 0.3305
Epoch 09 | Time: 230.9s
  Train Loss: 1.